In [1]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.config import list_physical_devices

import utils

In [2]:
list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
trainX, trainY, testX, testY = utils.load_dataset()

In [4]:
trainX.shape

(50000, 32, 32, 3)

In [5]:
def custom_vgglike():
    inputs = keras.Input(shape=(32,32,3))

    x = layers.Conv2D(32, (3,3), activation='relu', kernel_initializer='he_uniform', padding='same')(inputs)
    x = layers.Conv2D(32, (3,3), activation='relu', kernel_initializer='he_uniform', padding='same')(x)
    x = layers.MaxPool2D((2,2))(x)
    x = layers.Dropout(0.2)(x)

    x = layers.Conv2D(64, (3,3), activation='relu', kernel_initializer='he_uniform', padding='same')(x)
    x = layers.Conv2D(64, (3,3), activation='relu', kernel_initializer='he_uniform', padding='same')(x)
    x = layers.MaxPool2D((2,2))(x)
    x = layers.Dropout(0.2)(x)

    x = layers.Conv2D(128, (3,3), activation='relu', kernel_initializer='he_uniform', padding='same')(x)
    x = layers.Conv2D(128, (3,3), activation='relu', kernel_initializer='he_uniform', padding='same')(x)
    x = layers.MaxPool2D((2,2))(x)
    x = layers.Dropout(0.2)(x)

    x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu', kernel_initializer='he_uniform')(x)
    x = layers.Dropout(0.2)(x)
    
    outputs = layers.Dense(10, activation='softmax')(x)
    
    model = keras.Model(inputs=inputs, outputs=outputs, name="custom_vgglike")
    opt = keras.optimizers.SGD(learning_rate=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['categorical_accuracy', keras.metrics.AUC()])
    return model

In [7]:
callback = keras.callbacks.ModelCheckpoint('models/vgglike', monitor='val_loss', save_best_only=True)

In [8]:
model = custom_vgglike()
history = model.fit(x=trainX, y=trainY, batch_size=512, epochs=200, callbacks=callback, validation_data=(testX, testY))

- 2s 16ms/step - loss: 0.8250 - categorical_accuracy: 0.7067 - auc: 0.9610 - val_loss: 0.7844 - val_categorical_accuracy: 0.7242 - val_auc: 0.9646
Epoch 110/200
98/98 [==============================] - 2s 16ms/step - loss: 0.8179 - categorical_accuracy: 0.7079 - auc: 0.9617 - val_loss: 0.7720 - val_categorical_accuracy: 0.7302 - val_auc: 0.9657
INFO:tensorflow:Assets written to: models/vgglike/assets
Epoch 111/200
98/98 [==============================] - 2s 16ms/step - loss: 0.8182 - categorical_accuracy: 0.7117 - auc: 0.9618 - val_loss: 0.7697 - val_categorical_accuracy: 0.7291 - val_auc: 0.9659
INFO:tensorflow:Assets written to: models/vgglike/assets
Epoch 112/200
98/98 [==============================] - 2s 16ms/step - loss: 0.8150 - categorical_accuracy: 0.7117 - auc: 0.9618 - val_loss: 0.7796 - val_categorical_accuracy: 0.7260 - val_auc: 0.9652
Epoch 113/200
98/98 [==============================] - 2s 15ms/step - loss: 0.8077 - categorical_accuracy: 0.7141 - auc: 0.9625 - val_loss:

In [10]:
best_model = keras.models.load_model('models/vgglike')

In [11]:
best_model.evaluate(x=testX, y=testY)

313/313 [==============================] - 30s 50ms/step - loss: 0.6190 - categorical_accuracy: 0.7853 - auc: 0.9770


[0.6190477609634399, 0.7853000164031982, 0.9769587516784668]

In [ ]:
#architecture: https://machinelearningmastery.com/how-to-develop-a-cnn-from-scratch-for-cifar-10-photo-classification/